In [1]:
import os
import json
import requests
import datetime

#默认参数
default_config = {
    "channelid_youtube": {
        "YouTube": {
            "update_size": 1,
            "id": "UCBR8-60-B28hp2BmDPdntcQ",
            "title": "YouTube",
            "quality": "720",
            "last_size": 1,
            "media": "m4a"
        },
        "科学火箭叔": {
            "update_size": 15,
            "id": "UC8dD8v_uIA7aRzpqaCoIHKg",
            "title": "科学火箭叔",
            "quality": "1080",
            "last_size": 20,
            "media": "mp4"
        }
    }
}
expected_channelid = {
    "update_size": 1,
    "id": "UCBR8-60-B28hp2BmDPdntcQ",
    "title": "YouTube",
    "quality": "720",
    "last_size": 1,
    "media": "m4a"
}

In [2]:
def write_log(log):
    # 获取当前的具体时间
    current_time = datetime.datetime.now()
    # 格式化输出，只保留年月日时分秒
    formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
    # 打开文件，并读取原有内容
    try:
        with open("log.txt", "r") as file:
            contents = file.read()
    except FileNotFoundError:
        contents = ""
    # 将新的日志内容添加在原有内容之前
    new_contents = formatted_time + " " + log + "\n" + contents
    # 将新的日志内容写入文件
    with open("log.txt", "w") as file:
        file.write(new_contents)
    # 打印当前的具体时间（精确到秒）
    print(formatted_time, log)

In [3]:
# 检查当前文件夹中是否存在config.json文件
if not os.path.exists('config.json'):
    # 如果文件不存在，创建并写入默认字典
    with open('config.json', 'w') as file:
        json.dump(default_config, file)
    write_log("不存在配置文件，已新建，添加频道")
    config = default_config
else:
    # 如果文件存在，读取字典并保存到config变量中
    with open('config.json', 'r') as file:
        config = json.load(file)
    write_log("已读取配置文件")

2023-08-01 09:17:07 不存在配置文件，已新建，添加频道


In [4]:
folder_path = os.path.join(os.getcwd(), "channel_id")  # 构建文件夹路径
if not os.path.exists(folder_path):  # 判断文件夹是否存在
    os.makedirs(folder_path)  # 创建文件夹
    write_log(f"文件夹channel_id创建成功！")
else:
    write_log(f"文件夹channel_id已存在。")

2023-08-01 09:17:07 文件夹channel_id已存在。


In [5]:
url = "https://www.youtube.com/feeds/videos.xml?channel_id=UCBR8-60-B28hp2BmDPdntcQ"
response = requests.get(url)  # 发送GET请求获取网页内容
content = response.text  # 获取响应文本内容

file_path = os.path.join(folder_path, "UCBR8-60-B28hp2BmDPdntcQ.txt")  # 构建文件路径

with open(file_path, "w", encoding="utf-8") as file:
    file.write(content)

write_log("UCBR8-60-B28hp2BmDPdntcQ已更新")

2023-08-01 09:17:07 UCBR8-60-B28hp2BmDPdntcQ已更新
